<a href="https://colab.research.google.com/github/ekarla/ProjetoBemol/blob/main/mapa_vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Mapa de vendas por Loja</h2>
<p>Este projeto tem o objetivo de mostrar o total de vendas X dia X hora em cada unidade da Bemol.<br>
Para a contrução deste código estaremos utilizando as bibliotecas <i>"Pandas", "Glob" e "OS"</i>.
</p>

In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# Função pega a informação da coluna Hora e verifica o tamanho da informação e retorna os dois primeiros digitos
# se tamanho for igual 4 ou o primeiro digito se tamanho for igual a 3, caso contrario retorna 00

def trata_hora(hora):
    if(len(hora) == 4):
      return int(hora[0:2])
    elif(len(hora) == 3):
      return int(hora[0:1])
    else:
      return 00 

In [ ]:
#Definindo um intervalo para hora

def intervalo_hora(hora):
    if(hora >= 22):
      return 22
    elif(hora <= 8):
      return 8
    else:
      return hora

In [ ]:
#Criando uma lista com os codigos que representam tipos de compras.

def codigos_compra(base):
    filtro = base[(filtro_vendas['NÃO COMPRA'] != "X")] 
    filtro = filtro['CODIGO']
    return filtro   

In [ ]:
#Tratando a coluna hora e aplicando com a função "trata_hora"

def tratando_base_unidades(base):
    base['Hora'] = base['Hora'].astype(str)
    base['Hora'] = base.apply(lambda x: trata_hora(x.Hora), axis=1)
    return base

In [ ]:
#Realizando todos os cálculos de vendas. Subtraindo desconto e valor financiado

def calculando_valores(base_unidade):
    base_unidade['Valor +'] = (base_unidade['Qtde. Prestações'] * base_unidade['Vlr. Prestações']) + base_unidade['Vlr. Entrada'] + base_unidade['Vlr. Juros'] 
    base_unidade['Valor -'] = base_unidade['Valor Desconto'] + base_unidade['Vlr. Financiamento']
    base_unidade['Valor Venda'] = base_unidade['Valor +'] - base_unidade['Valor -']
    mapa = base_unidade[['Centro','Data','Hora','Qtde. Prestações','Valor +','Valor -','Valor Venda']]
    return mapa


<h3>Leitura dos arquivos<h3>

In [ ]:
path = r'/Users/16116/Google Drive/mapa_vendas/bases_lojas' # passando o caminho onde estão os arquivos base
files= glob.glob(os.path.join(path, "*.XLSX")) 
filtro_vendas = pd.read_excel("base_operacoes.xlsx",sheet_name="TRANSACOES")

<h3>Fazendo a Mágica</h3>
<p> Nesta célula, são chamados as funções que realiza o tratamento dos dados<br>
    <i>tratando_base_unidades()</i>. Realizamos o tratamento da coluna "Hora" e da
    coluna "Data".<br> E fazemos a concatenação dos arquivos de cada unidade com o 
    comando <i>append</i>
</p>

In [ ]:
lista_vendas = []
filtro = codigos_compra(filtro_vendas)
for i in range(len(files)):
    arquivo_unidade = pd.read_excel(files[i])
    vendas_unidade = tratando_base_unidades(arquivo_unidade)
    vendas_unidade = arquivo_unidade[(arquivo_unidade['ID Transação Caixa'].isin(filtro))].copy()
    vendas_unidade['Qtde. Prestações'] = vendas_unidade['Qtde. Prestações'].replace(0,1)
    vendas_unidade['Hora'] = vendas_unidade['Hora'].astype(int)
    vendas_unidade['Hora'] = vendas_unidade.apply(lambda x: intervalo_hora(x.Hora), axis=1)
    vendas_unidade['Data'] = pd.to_datetime(vendas_unidade['Data']).dt.day
    mapa_vendas = calculando_valores(vendas_unidade)
    lista_vendas.append(mapa_vendas)
    

<h3>Criação de Dataframe Mapa_vendas_lojas</h3>
<p>Nesta celula realizamos a criação do dataframe com todas as informações de todas as unidades.<br>
Após a criação do dataframe é realizado um agrupamento das informações com a função <i>groupby</i>, usando como parametro:<br>
"Centro", "Data", "Hora" e realizando a soma dos valores de venda por dia e hora.
</p>

In [ ]:
df_vendas = pd.DataFrame()
for i in range(len(lista_vendas)):
    df_vendas = pd.concat([df_vendas,lista_vendas[i]])
    
base_geral = df_vendas.groupby(['Centro','Data','Hora'],as_index=False).agg({'Valor Venda': 'sum'})
#base_geral

<h3>Salvando o arquivo</h3>
<p>Após realizar o tratamento e todos os calculos solicitados, salvamos o arquivo no formato <i>.txt</i>
para utilizarmos posteriormente.
</p>

In [ ]:
base_geral.to_excel("base_vendas_lojas.xlsx",index=False)